<a href="https://colab.research.google.com/github/agusnieto77/taller_pict_2019/blob/main/Colabs/ACEP_etiquetado_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from google.colab import data_table
#data_table.enable_dataframe_formatter()
data_table.disable_dataframe_formatter()

In [21]:
df_head = pd.read_csv('/content/drive/MyDrive/R/PROYECTOS/taller_pict_2019/data/selec_notas_etiq_s.csv').head(5)
df_tail = pd.read_csv('/content/drive/MyDrive/R/PROYECTOS/taller_pict_2019/data/selec_notas_etiq_s.csv').tail(5)
df = pd.concat([df_head, df_tail])
df

,id_usuario,id_nota,tbn,es_conflicto,es_principal
0,16,13068,El costo de los alquileres subió el 30% en Mar...,N,NaN
1,16,10026,La Justicia marplatense rechazó cautelar docen...,N,NaN
2,7,13002,"Por el Día de la Madre, extendieron el horario...",N,NaN
3,7,10526,Piden que Randazzo lidere un espacio peronista...,N,NaN
4,16,10825,Invasión de mosquitos: hasta cuándo durará y l...,N,NaN
95,13,12738,Organizaciones sociales cortan la autovía 2 en...,Y,Y
96,16,10634,"Sin la marcha del 24, anunciaron las actividad...",Y,Y
97,16,12492,"Salvarezza, a los investigadores: &#8220;Estam...",Y,Y
98,16,10835,La UTA llegó a un acuerdo con los empresarios ...,Y,Y
99,16,12189,Hay paro de colectivos en Mar del Plata por ti...,Y,Y


In [3]:
from transformers import pipeline
import pandas as pd

In [ ]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")

In [8]:
candidate_labels = ["protesta", "conflicto", "salud", "clima", "economía", "turismo", "política", "cultura", "deportes", "misceláneas"]

In [ ]:
results = []

for index, row in df.iterrows():
    id_nota = row['id_nota']
    sequence_to_classify = row['tbn']
    output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
    result = {
        'id_nota': id_nota,
        'sequence': sequence_to_classify,
        'labels': output['labels'],
        'scores': output['scores']
    }
    results.append(result)

In [22]:
result_df = pd.DataFrame(results)
result_df

,id_nota,sequence,labels,scores
0,13068,El costo de los alquileres subió el 30% en Mar...,"[conflicto, economía, política, misceláneas, c...","[0.41947588324546814, 0.3315608501434326, 0.08..."
1,10026,La Justicia marplatense rechazó cautelar docen...,"[salud, protesta, misceláneas, clima, política...","[0.7368423938751221, 0.07185105234384537, 0.05..."
2,13002,"Por el Día de la Madre, extendieron el horario...","[misceláneas, economía, política, salud, confl...","[0.46449044346809387, 0.38349655270576477, 0.1..."
3,10526,Piden que Randazzo lidere un espacio peronista...,"[política, misceláneas, protesta, conflicto, s...","[0.5040966868400574, 0.31062063574790955, 0.12..."
4,10825,Invasión de mosquitos: hasta cuándo durará y l...,"[salud, misceláneas, clima, conflicto, polític...","[0.6787046194076538, 0.19540242850780487, 0.04..."
5,12738,Organizaciones sociales cortan la autovía 2 en...,"[protesta, conflicto, política, salud, miscelá...","[0.3381192684173584, 0.22510391473770142, 0.16..."
6,10634,"Sin la marcha del 24, anunciaron las actividad...","[misceláneas, protesta, salud, cultura, políti...","[0.4754965305328369, 0.24362362921237946, 0.12..."
7,12492,"Salvarezza, a los investigadores: &#8220;Estam...","[protesta, política, conflicto, economía, misc...","[0.6220555305480957, 0.23284609615802765, 0.07..."
8,10835,La UTA llegó a un acuerdo con los empresarios ...,"[conflicto, economía, misceláneas, política, p...","[0.48525470495224, 0.33911699056625366, 0.0943..."
9,12189,Hay paro de colectivos en Mar del Plata por ti...,"[conflicto, protesta, misceláneas, política, e...","[0.5068315267562866, 0.3565787374973297, 0.117..."


In [14]:
data_norm = []

for _, row in result_df.iterrows():
    id_nota = row['id_nota']
    sequence = row['sequence']
    labels = row['labels']
    scores = row['scores']
    for label, score in zip(labels, scores):
        data_norm.append({
            'id_nota': id_nota,
            'sequence': sequence,
            'labels': label,
            'scores': score
        })

In [23]:
df_norm = pd.DataFrame(data_norm)
df_norm

,id_nota,sequence,labels,scores
0,13068,El costo de los alquileres subió el 30% en Mar...,conflicto,0.419476
1,13068,El costo de los alquileres subió el 30% en Mar...,economía,0.331561
2,13068,El costo de los alquileres subió el 30% en Mar...,política,0.089201
3,13068,El costo de los alquileres subió el 30% en Mar...,misceláneas,0.075712
4,13068,El costo de los alquileres subió el 30% en Mar...,cultura,0.032127
...,...,...,...,...
95,12189,Hay paro de colectivos en Mar del Plata por ti...,cultura,0.001027
96,12189,Hay paro de colectivos en Mar del Plata por ti...,salud,0.000849
97,12189,Hay paro de colectivos en Mar del Plata por ti...,turismo,0.000390
98,12189,Hay paro de colectivos en Mar del Plata por ti...,deportes,0.000316
